# Добавляем память в OpenAI Functions Agent

Этот ноутбук показывает, как можно добавить память в OpenAI Functions agent. Также здесь показано, как можно использовать этот функционал для решения бизнес-задач.

In [1]:
import os
from langchain import (
    LLMMathChain,
    OpenAI,
    SerpAPIWrapper
)
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [2]:
# os.environ["OPENAI_API_KEY"] = "..."
llm = ChatOpenAI(temperature=0, model="gpt-4")

In [3]:
system = "Ты бот-продавец телефонов. Твоя задача продать телефон пользователю, получив от него заказ. Необходимые для заказа данные уточняй у пользователя."

stuff_database = [
    {"name": "iPhone 8 mini", "price": 300, "memory": 64, "ram": 4, "camera": 8, "description": "Самая дешевая модель iPhone"},
    {"name": "iPhone 14", "price": 1000, "memory": 512, "ram": 12, "camera": 12, "description": "Телефон будущего, уже сегодня!"},
    {"name": "Samsung Galaxy S23", "price": 900, "memory": 256, "ram": 12, "camera": 108, "description": "Камера такая острая, что сможет увидеть даже ваши ошибки"},
    {"name": "Google Pixel 7", "price": 850, "memory": 128, "ram": 8, "camera": 16, "description": "Для тех, кто хочет получить стоковый Android и хорошие фотки"},
    {"name": "OnePlus 9T", "price": 700, "memory": 128, "ram": 8, "camera": 48, "description": "Зарядка быстрее, чем ваш кофе"},
    {"name": "Xiaomi Mi 12", "price": 600, "memory": 128, "ram": 6, "camera": 64, "description": "Бюджетный флагман для ценителей вкуса"},
    {"name": "Sony Xperia 3", "price": 1100, "memory": 256, "ram": 12, "camera": 20, "description": "Телефон для тех, кто скучал по кнопке для камеры"},
    {"name": "Huawei P60", "price": 800, "memory": 128, "ram": 8, "camera": 50, "description": "Для любителей ночной съемки и без Google Play"},
    {"name": "Nokia 10 PureView", "price": 750, "memory": 128, "ram": 6, "camera": 48, "description": "Nokia вернулась, и у неё есть змейка!"},
    {"name": "LG Velvet 2", "price": 650, "memory": 128, "ram": 8, "camera": 32, "description": "Потому что жизнь хороша"},
    {"name": "Asus ROG Phone 6", "price": 1000, "memory": 512, "ram": 16, "camera": 64, "description": "Играй как профи, заряжай как новичок"},
    {"name": "Motorola Edge Plus", "price": 700, "memory": 128, "ram": 8, "camera": 108, "description": "Край к краю, пиксель к пикселю"},
    {"name": "Realme X4 Pro", "price": 450, "memory": 128, "ram": 8, "camera": 48, "description": "Экономия без потерь в качестве"},
    {"name": "Oppo Find X4", "price": 900, "memory": 256, "ram": 12, "camera": 50, "description": "Найди X, но без математики"},
    {"name": "BlackBerry Secure", "price": 1200, "memory": 128, "ram": 8, "camera": 12, "description": "Для тех, кто ещё помнит, что такое физическая клавиатура"},
    {"name": "Fairphone 4", "price": 500, "memory": 64, "ram": 4, "camera": 12, "description": "Этичный выбор для заботливого потребителя"}
]

In [4]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [5]:
from langchain.tools import tool
from typing import Dict, List

@tool
def get_all_phone_names(txt: str) -> List[str]:
    """Возвращает названия моделей всех телефонов через запятую"""
    # Print with green color
    print("\033[92m" + "Bot requested get_all_phone_names" + "\033[0m")
    return [stuff["name"] for stuff in stuff_database]

@tool
def get_phone_data_by_name(name: str) -> Dict:
    """Возвращает цену, характеристики и описание телефона по строгому названию модели"""
    print("\033[92m" + f"Bot requested get_phone_data_by_name({name})" + "\033[0m")
    for stuff in stuff_database:
        if stuff["name"] == name:
            return stuff
    return None

@tool
def create_order(model_name: str, phone_number: str) -> str:
    """Создает новый заказ на телефон model_name. Для заказа нужно узнать у пользователя номер телефона phone_number
    
    parameters:
        model_name: название модели телефона
        phone_number: номер телефона пользователя для связи

    returns:
        OK в случае успешного заказа
    """
    if not model_name:
        return "Incorrect name parameter"
    if not phone_number:
        return "Incorrect phone parameter"
    print("\033[92m" + f"Bot requested create_order({model_name}, {phone_number})" + "\033[0m")
    return "OK"

In [6]:
agent_chain = initialize_agent(
    [get_all_phone_names, get_phone_data_by_name, create_order],
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=False,
    agent_kwargs=agent_kwargs,
    memory=memory,
    agentArgs={
        "systemMessage": system
    },
)

In [11]:
print("\033[94m" + "System: " + system + "\033[0m")
while True:
    q = input("User: ")
    if q == "":
        break
    print(f"User: {q}")
    resp = agent_chain.run(input=q)
    print(f"\033[94mBot: {resp}\033[0m")

System: Ты бот-продавец телефонов. Твоя задача продать телефон пользователю, получив от него заказ. Необходимые для заказа данные уточняй у пользователя.
User: Здравствуйте. Я бы хотел купить айфон
Bot requested get_all_phone_names
Bot: У нас в наличии следующие модели iPhone:

1. iPhone 8 mini
2. iPhone 14

Пожалуйста, выберите модель, которую вы хотите приобрести.
User: А сколько они стоят?
Bot requested get_phone_data_by_name(iPhone 8 mini)
Bot requested get_phone_data_by_name(iPhone 14)
Bot: Вот информация о ценах и характеристиках доступных моделей iPhone:

1. iPhone 8 mini - $300. Это самая дешевая модель iPhone с 64GB памяти, 4GB оперативной памяти и 8MP камерой.

2. iPhone 14 - $1000. Это телефон будущего, уже сегодня! Он имеет 512GB памяти, 12GB оперативной памяти и 12MP камерой.

Пожалуйста, выберите модель, которую вы хотите приобрести.
User: Что нужно, чтобы купиь у вас тот, что подешелве?
Bot: Для того чтобы купить iPhone 8 mini, мне нужно знать ваш номер телефона для связ